In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
from random import random
import math

In [5]:
import requests
import urllib3
import datetime
urllib3.disable_warnings()

# Generate dates
base = datetime.datetime.today()
date_list = [base - datetime.timedelta(days=x) for x in range(2*365+5)]
dates = []
for dt in date_list:
    if dt.weekday() < 5:
        dates.append(dt.strftime("%Y%m%d"))

# Tickers
tickers = ['SPY','XLC','XLY','XLP','XLE','XLF','XLV','XLI','XLB','XLRE','XLK','XLU']

# CSV Columns
cols = ['Ticker','Date','CallVol','PutVol','P/C']
df = pd.DataFrame(columns=cols)

# Main loop
for ticker in tickers:
    for date in dates:
        a = [ticker, date]
        for PC in ['C','P']:
            r = requests.get(f"https://marketdata.theocc.com/volume-query?reportDate={date}&format=csv\
            &volumeQueryType=O&symbolType=O&symbol={ticker}&reportType=D&accountType=ALL&productKind=ALL\
            &porc={PC}", verify=False, allow_redirects=False)
            total = -1
            for line in r.iter_lines(chunk_size=1024):
                # Skip first line
                if total == -1:
                    total += 1
                    continue
                if line:
                    lineStr = str(line)
                    commaIndex = lineStr.find(',')
                    total += int(lineStr[2:commaIndex])
            a.append(total)
        if a[-2] == 0:
            continue
            a.append(0)
        else:
            a.append(a[-1]/a[-2])
        dfT = pd.DataFrame([a], columns=cols)
        df = df.append(dfT, ignore_index=True)
df.to_csv('OptionVolume.csv', index=False, header=False)